# 네이버 맛집별 리뷰 크롤링
### 깃헙 주소: github.com/jw0831

In [222]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd
from tqdm.notebook import tqdm
import random

In [474]:
# 크롬브라우저 시작!
driver = webdriver.Chrome("/Users/jinwon-kim/Developer/chromedriver")

In [419]:
# 네이버 주소페이지로 이동
driver.get('https://m.naver.com/')
time.sleep(1)

In [454]:
# 버튼 옵션 테스팅
btns=['_1_hm2','es3Ot','_2EZQu']
restaurant_btns = driver.find_elements_by_xpath("//span[@class='{}']".format(btns[1]))
len(restaurant_btns)
restaurant_btns[0].text

'A 아쿠아플라넷 제주'

In [456]:
#식당이름 클릭전 식당이름과 클릭하는 식당의 이름이 같은지 검증
btns=['_1_hm2','es3Ot','_2EZQu']
collect_failed=[]
try:
    for btn in btns:
        restaurant_btns = driver.find_elements_by_xpath("//span[@class='{}']".format(btn))
        if len(restaurant_btns) == 1:
            restaurant_btns.click()
            time.sleep(1.5)
            break
        else:
            for restaurant_btn in restaurant_btns:
                #print(restaurant_btn.text)
                if restaurant in restaurant_btn.text: #자동시스템에서 name 으로 변경
                    restaurant_btn.click()
                    time.sleep(1.5)
                    break
                    
                    
#     print("ok")
except:
    collect_failed.append(name)
#     print("no")

ok


In [447]:
restaurant_btn.text

'E 휘닉스 제주 섭지코지'

In [412]:
restaurant_btn

[<selenium.webdriver.remote.webelement.WebElement (session="e80cd2355868445bd5a7f64f14d94f88", element="92a5b3da-23a6-4dda-af47-fbf1f0fa7014")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e80cd2355868445bd5a7f64f14d94f88", element="d1f2a841-a210-4a7c-8908-0f8d81e2d4bd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e80cd2355868445bd5a7f64f14d94f88", element="39891cc3-b6c9-4243-8761-f1635d2d5e0f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e80cd2355868445bd5a7f64f14d94f88", element="226afb5a-8492-45a0-9b8b-a721811c85a3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e80cd2355868445bd5a7f64f14d94f88", element="afe3194c-40b1-4cb7-b795-313112b297e8")>]

In [389]:
# 추출할 맛집 리스트 불러오기 (리스트 크롤 by 조원우|이찬주)
mango_df=pd.read_excel("mango_crawling_df.xlsx", index_col=0)
mango_df.head()

,가게명,주소,점수,찜,음식 분야,해시태그
0,상춘재,제주특별자치도 제주시 조천읍 선진길 26,4.7,"4,703",한정식 / 백반 / 정통 한식,"['#비빔밥', '#제주도', '#2018 인기', '#Bibimbap', '#20..."
1,남양수산,제주특별자치도 서귀포시 성산읍 고성동서로56번길 11,4.6,"5,008",해산물 요리,"['#횟집', '#2020망고플레이트인기맛집', '#한식', '#2018 인기', ..."
2,선흘곶,제주특별자치도 제주시 조천읍 동백로 102,4.6,"3,210",한정식 / 백반 / 정통 한식,"['#2020망고플레이트인기맛집', '#아이와함께', '#제주 함덕', '#김녕',..."
3,스시호시카이,제주특별자치도 제주시 오남로 90,4.5,"1,641",회 / 스시,"['#제주도', '#2018 인기', '#스시오마카세', '#초밥', '#Jeju'..."
4,미영이네식당,제주특별자치도 서귀포시 대정읍 하모항구로 42,4.5,"2,748",해산물 요리,"['#고등어', '#2020망고플레이트인기맛집', '#제주 모슬포', '#화순', ..."


In [457]:
# 리뷰 추출할 식당이름 시리즈로 불러오기
restaurants=pd.Series(mango_df['가게명'])[49:]
restaurants.head()

49           섭지코지로
50    원더먼트제주 (휴업중)
51            만선식당
52            무거버거
53        포도호텔레스토랑
Name: 가게명, dtype: object

In [458]:
#데이터 저장소
data=[]

In [459]:
#crawling start
jeju = "제주 "
SCROLL_PAUSE_TIME = 1.5
collect_failed=[]
btns=['_1_hm2','es3Ot','_2EZQu']
for restaurant in restaurants:
    #main page 불러오기
    driver.get('https://m.naver.com/')
    time.sleep(1)
    name=restaurant
    # 검색창에 검색어 입력하기
    driver.find_element_by_xpath("//input[@id='MM_SEARCH_FAKE']").click() #fake 해제 시킴 (성공)
    time.sleep(1)
    search_box = driver.find_element_by_css_selector("input.sch_inp") #진짜 검색창


    search_box.send_keys(jeju+name)  #jeju가 앞인지 뒤인지 파악해보기 + 식당
    time.sleep(1.0)
    
    # 검색버튼 누르기
    search_box.send_keys(Keys.ENTER)
    time.sleep(2.0)
    # 식당이름 클릭
    try:
        for btn in btns:
            restaurant_btns = driver.find_elements_by_xpath("//span[@class='{}']".format(btn))
            if len(restaurant_btns) == 1:
                restaurant_btns.click()
                time.sleep(1.5)
                break
            else:
                for restaurant_btn in restaurant_btns:
                    #print(restaurant_btn.text)
                    if name in restaurant_btn.text: #자동시스템에서 name 으로 변경
                        restaurant_btn.click()
                        time.sleep(1.5)
                        break
        # 방문자리뷰 클릭
        driver.find_element_by_xpath("//span[@class='_1Y6hi']").click()
        time.sleep(1.5)

        #별 평점
        star_score=driver.find_element_by_css_selector("div.hRJcF")
        star_score=star_score.text.split("/")[0]
        star_score

        #별 참여 개수
        star_count=driver.find_element_by_css_selector("div._1XXGh").text.split("명")[0].split("(")[1]
        star_count

        #리뷰 "더보기 클릭"
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            try:
                #스크롤 맨밑으로 내리기
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(SCROLL_PAUSE_TIME) #1.5초 지연
                driver.find_element_by_xpath("//a[@class='_3iTUo']").click() #더보기 클릭
            except:
                 #종료 조건 (더보기 클릭이 더이상 되지않을때 종료)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
            last_height = new_height

        #모든 리뷰 추출
        reviews = driver.find_elements_by_css_selector("div.PVBo8 > a > span.WoYOw")
        #리뷰 개수
        review_count=len(reviews) 

        review_data = []
        for review in reviews:
            review_data.append(review.text)

        temp_data={'name':name, 
                'star_score':star_score, 
                'star_count':star_count, 
                'review_count':review_count,
                'review_data':review_data}

        data.append(temp_data)
    except:
        collect_failed.append(name) #실패할경우 셀프 크롤링을 위해서 맛집이름 저장
#crawling finished
naver_review=pd.DataFrame(data)
naver_review.to_csv("naver_review_crawling.csv", index=False) #저장


In [402]:
#저장 실패했을경우 = 셀프저장
naver_review=pd.DataFrame(data)
naver_review.to_csv("naver_review_crawling_fc.csv", index=False) #저장

___

# 실패한것들 셀프 크롤링

In [475]:
driver.get('https://m.naver.com/')
time.sleep(1)

In [476]:
cf_temp_data=[] #모으는동안 처음한번실행후 제실행 금지입니다.

In [521]:
# 지정 추출 (오류만 따로)
cf = mango_df.iloc[145,0]
cf

'안거리밖거리'

In [477]:
# 크롤링 실패한 맛집리스트
cf = pd.Series(collect_failed)
cf

0     원더먼트제주 (휴업중)
1             제주맥주
2       이니스프리제주하우스
3             광동식당
4             삼영식당
5             풍림다방
6           941헤르츠
7             요요무문
8             우도왕자
9               보엠
10       중앙통닭 마농치킨
11           랜디스도넛
12         커피박물관바움
dtype: object

In [523]:
jeju = "제주 "
# 검색창
driver.get('https://m.naver.com/')
time.sleep(1)
# 검색창에 검색어 입력하기
driver.find_element_by_xpath("//input[@id='MM_SEARCH_FAKE']").click() #fake 해제 시킴 (성공)
time.sleep(1)
search_box = driver.find_element_by_css_selector("input.sch_inp") #진짜 검색창

name=cf #오류만 따로 할경우
# name=cf[0]
#식당명 변경 숫자 + 하기 (셀프)
search_box.send_keys(jeju+name)  #jeju가 앞인지 뒤인지 파악해보기 + 식당
time.sleep(1.0)

# 검색버튼 누르기
search_box.send_keys(Keys.ENTER)
time.sleep(1.5)

In [524]:
# 식당클릭 (셀프)

# 방문자리뷰 클릭
# driver.find_element_by_xpath("//span[@class='_1Y6hi']").click()
# time.sleep(1.5)

#별 평점
star_score=driver.find_element_by_css_selector("div.hRJcF")
star_score=star_score.text.split("/")[0]
star_score
#별 참여 개수
star_count=driver.find_element_by_css_selector("div._1XXGh").text.split("명")[0].split("(")[1]
star_count


# 셀프인 경우
# star_score = 4.44
# star_count = 104

#리뷰 "더보기 클릭"
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    try:
        #스크롤 맨밑으로 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME) #1.5초 지연
        driver.find_element_by_xpath("//a[@class='_3iTUo']").click() #더보기 클릭
    except:
         #종료 조건 (더보기 클릭이 더이상 되지않을때 종료)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
    last_height = new_height

#모든 리뷰 추출
reviews = driver.find_elements_by_css_selector("div.PVBo8 > a > span.WoYOw")
#리뷰 개수
review_count=len(reviews) 

review_data = []
for review in reviews:
    review_data.append(review.text)

temp_data={'name':name, 
        'star_score':star_score, 
        'star_count':star_count, 
        'review_count':review_count,
        'review_data':review_data}

cf_temp_data.append(temp_data)


In [525]:
pd.DataFrame(cf_temp_data)

,name,star_score,star_count,review_count,review_data
0,제주맥주,4.53,"1,058",960,"[근처에 있다면 가볼만한집, 좋아요, 좋아요, 양조장 투어 재밌어요 맥주도 맛있고 ..."
1,이니스프리제주하우스,4.49,"1,599",543,"[생각보다 이뻐요 가보세요, 좋아요! 주변도 너무 이쁘고!, 늘 좋아요^^, 케이크..."
2,광동식당,4.37,22,15,"[레전드, 별점이 5개밖에없는게 통탄스러워요.. 사장님 넘 후하시고 고기 무한리필에..."
3,삼영식당,4.53,28,19,"[진짜 현지인 맛집입니다 몸국추천합니다, 몸국 맛있어요~, 굿, 현지인 맛집느낌, ..."
4,풍림다방,4.32,826,370,[다 좋은데 너무 복잡해요 밖에 테이블을 더 놨으면 좋겠어요!!! 간이 의자라도 있...
5,요요무문,4.44,104,45,"[바다뷰 예뻐요~ 당근쥬스 완전 맛있!, 그릇 이뿜 바다 이뿜, 당근주스 현재 90..."
6,우도왕자,3.9,45,17,"[사장님 친절하시고 아이스크림도 맛있어요~~두번 먹었어요🧡, #검멀레해변 앞에있음 ..."
7,보엠,4.36,60,22,"[버터프레첼, 올리브치아바타 아몬드크루아상 전화픽업예약했어요. 예약되서 편하고 좋네..."
8,중앙통닭 마농치킨,4.24,469,168,"[맛있어요~, 맛있어여 ㅋ, 양이 우선 많고 맛도 괜찮아요, 항상 맛있게 먹고있어요..."
9,랜디스도넛,4.39,646,286,"[너무막잇오ㅠㅠ, 도넛 맛있어요, 크리스피랑 달라요.크리스피는 사르르 녹는느낌이면 ..."


In [526]:
naver_review=pd.DataFrame(cf_temp_data)
naver_review.to_csv("naver_review_crawling_fc.csv", index=False) #저장

___
## 여기까지 실패한 리뷰 셀프 추출(완료)
___

In [527]:
#브라우저 종료
driver.close()

___
## 테스팅 코드

In [370]:
# 검색창에 검색어 입력하기
# driver.find_element_by_css_selector("span.sch_inpw_in").click() #fake 해제 시킴
# driver.find_element_by_css_selector("//input[@id='MM_SEARCH_FAKE']").click() #fake 해제 시킴
driver.find_element_by_xpath("//input[@id='MM_SEARCH_FAKE']").click() #fake 해제 시킴 (성공)
time.sleep(1)
search_box = driver.find_element_by_css_selector("input.sch_inp") #진짜 검색창

jeju = "제주 "
search_box.send_keys(jeju+"88돼지")  #jeju가 앞인지 뒤인지 파악해보기 + 식당
time.sleep(2.0)

In [371]:
# 검색버튼 누르기
search_box.send_keys(Keys.ENTER)
time.sleep(1.5)

In [372]:
# 식당이름 클릭
try:
    driver.find_element_by_xpath("//a[@class='_1_hm2']").click()
    time.sleep(1.5)
except:
    driver.find_element_by_xpath("//span[@class='es3Ot']").click()

In [373]:
# 방문자리뷰 클릭
driver.find_element_by_xpath("//span[@class='_1Y6hi']").click()
time.sleep(1.5)

In [374]:
#별 평점
star_score=driver.find_element_by_css_selector("div.hRJcF")
star_score=star_score.text.split("/")[0]
star_score

'4.39'

In [375]:
#별 참여 개수
star_count=driver.find_element_by_css_selector("div._1XXGh").text.split("명")[0].split("(")[1]
star_count

'99'

#리뷰 "더보기 클릭"
driver.find_element_by_xpath("//a[@class='_3iTUo']").click()

In [376]:
#리뷰 "더보기 클릭"
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    try:
        #스크롤 맨밑으로 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME) #1.5초 지연
        driver.find_element_by_xpath("//a[@class='_3iTUo']").click() #더보기 클릭
    except:
         #종료 조건 (더보기 클릭이 더이상 되지않을때 종료)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
    last_height = new_height

In [377]:
#모든 리뷰 추출
reviews = driver.find_elements_by_css_selector("div.PVBo8 > a > span.WoYOw")

In [378]:
review_count=len(reviews) #리뷰 개수
review_count

40

reviews[39].text #테스트

In [379]:
review_data = []
for review in reviews:
    review_data.append(review.text)
# review_data

reviews=driver.find_element_by_xpath("//span[@class='WoYOw']")

In [ ]:
data=[]
restaurant={#'name':name, 
            'star_score':star_score, 
            'star_count':star_count, 
            'review_count':review_count,
            'review_data':review_data}

data.append(restaurant)

In [380]:
restaurant={#'name':name, 
            'star_score':star_score, 
            'star_count':star_count, 
            'review_count':review_count,
            'review_data':review_data}

data.append(restaurant)

In [396]:
pd.DataFrame(data)

""


#crawling start
jeju = "제주 "
SCROLL_PAUSE_TIME = 1.5
collect_failed=[]
btns=['_1_hm2','es3Ot','_2EZQu']
for restaurant in restaurants:
    #main page 불러오기
    driver.get('https://m.naver.com/')
    time.sleep(1)
    name=restaurant
    # 검색창에 검색어 입력하기
    driver.find_element_by_xpath("//input[@id='MM_SEARCH_FAKE']").click() #fake 해제 시킴 (성공)
    time.sleep(1)
    search_box = driver.find_element_by_css_selector("input.sch_inp") #진짜 검색창


    search_box.send_keys(jeju+name)  #jeju가 앞인지 뒤인지 파악해보기 + 식당
    time.sleep(2.0)
    
    # 검색버튼 누르기
    search_box.send_keys(Keys.ENTER)
    time.sleep(1.5)
    # 식당이름 클릭
    try:
        driver.find_element_by_xpath("//a[@class='_1_hm2']").click()
        time.sleep(1.5)
    except:
        try:
            driver.find_element_by_xpath("//span[@class='es3Ot']").click() #식당 리스트가 여러개일경우 가장위에것 클릭
            time.sleep(1.5)
        except:
            driver.find_element_by_xpath("//span[@class='_2EZQu']").click() #식당 리스트가 여러개일경우 가장위에것 클릭
            time.sleep(1.5)
    # 방문자리뷰 클릭
    driver.find_element_by_xpath("//span[@class='_1Y6hi']").click()
    time.sleep(1.5)
    
    #별 평점
    star_score=driver.find_element_by_css_selector("div.hRJcF")
    star_score=star_score.text.split("/")[0]
    star_score
    
    #별 참여 개수
    star_count=driver.find_element_by_css_selector("div._1XXGh").text.split("명")[0].split("(")[1]
    star_count
    
    #리뷰 "더보기 클릭"
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        try:
            #스크롤 맨밑으로 내리기
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME) #1.5초 지연
            driver.find_element_by_xpath("//a[@class='_3iTUo']").click() #더보기 클릭
        except:
             #종료 조건 (더보기 클릭이 더이상 되지않을때 종료)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
        last_height = new_height
        
    #모든 리뷰 추출
    reviews = driver.find_elements_by_css_selector("div.PVBo8 > a > span.WoYOw")
    #리뷰 개수
    review_count=len(reviews) 
    
    review_data = []
    for review in reviews:
        review_data.append(review.text)
    
    temp_data={'name':name, 
            'star_score':star_score, 
            'star_count':star_count, 
            'review_count':review_count,
            'review_data':review_data}

    data.append(temp_data)
#crawling finished
naver_review=pd.DataFrame(data)
naver_review.to_csv("naver_review_crawling.csv", index=False) #저장


___
### 연습코드

In [ ]:
에구 저도 해봐야 할 것 같아요 ㅠㅠ def crawling(self): 코드가 쭈욱 나오고~~ json.loads(data.text, encoding='utf-8') self.data['컬럼명"] 이런식으로 추출하거든요
                                                                                                        
self.get_json_value("컬럼명")

In [ ]:
self.get_json_value("컬럼명") 이런식도 가능하고요 json 예를 찾아보는게 좋을것 같아요 ㅠㅠ 동적페이지는 사실 ajax 를 따로 공부해야하는데\ 우리 과정하고는 별개라서

In [84]:
# 동적페이지 예제
import json

jsonString = """{
    "number":[{"num":0},{"num":1},{"num":2}],
    "fruits":[{"fruit":"apple"},{"fruit":"banana"},{"fruit":"pear"}]
}"""

jsonObj = json.loads(jsonString)

print(jsonObj.get("number"))
print(jsonObj.get("number")[1])
print(jsonObj.get("number")[1].get("num"))
print(jsonObj.get("number")[1].get("num") + jsonObj.get("number")[2].get("num"))
print(jsonObj.get("fruits")[1])

[{'num': 0}, {'num': 1}, {'num': 2}]
{'num': 1}
1
3
{'fruit': 'banana'}


In [ ]:
elif '블로그 리뷰' in info:
    headers2={
        "Referer":"https://store.naver.com/restaurants/detail?",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"
    }

    params2 = {
        "exclude":"",
        "businessId":place_a_id,
        "category":"restaurant",
        "start": 1,
        "display": 50
    }

    naver_blog_url = "https://store.naver.com/sogum/api/fsasReviews"
    res = requests.get(naver_blog_url,headers=headers2, params=params2)
    blog_dict = json.loads(res.text)
    print('--블로그 리뷰 : ',len(blog_dict['items']))

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html,'lxml')

content = soup.select('ul:nth-child(1) > div > li > div > div.C7I1f > div.C4VMK > span')[0].text
narration = re.findall("[가-힣ㄱ-ㅎ\D]+\s+[^#]+", content)

tags = re.findall("#[\w,가-힣ㄱ-ㅎ]+",content)
comments = soup.select('ul:nth-child(n+2) > div > li > div > div.C7I1f > div.C4VMK > span')
# comments
# temp_comments=[]
# for comment in comments:
#     temp_comments.append(comment.text)
# temp_comments
# content
narration

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    try:
        content = soup.select('div.C4VMK>span')[0].text
    except:
        content =''
    #설명만 가져오기
    try:
        narration = re.findall("[가-힣ㄱ-ㅎ\D]+\s+[^#]+", content)
    except:
        narration = ''
    #해시태그만 가져오기
    try:
        tags = re.findall("#[\w,가-힣ㄱ-ㅎ]+",content)
    except:
        tags=''
    
    #작성일자 가져오기
    try:
        date=soup.select("time._1o9PC.Nzb55")[0]['datetime'][:10]
    except:
        date =''
    try:
        place = soup.select('div.M30cS')[0].text # 위치정보를 알려주는 코드
    except:
        place=''
    #댓글 가져오기 (감성분석 사용)
    temp_comments=[]
    try:
        comments = soup.select('ul:nth-child(n+2) > div > li > div > div.C7I1f > div.C4VMK > span')
        for comment in comments:
            temp_comments.append(comment.text)
    except:
        comments = ''
#     data =[tags,date,place]
#     data={'tags':tags,'date':[date],'place':[place]}
    data={'narration':narration,'tags':tags,'date':date,'place':place,'comments':temp_comments}

    return data
get_content(driver) #감성분석할때 제외할 것 "\xa0"

In [ ]:
def moveNext(driver):
    try:
        right=driver.find_element_by_css_selector("a.coreSpriteRightPaginationArrow")
        right.click()
        time.sleep( random.uniform(2,7))
    except: 
        driver.close()
moveNext(driver)



In [ ]:
try:
    results=[]
    for i in tqdm(range(10000)):
        results.append(get_content(driver))
        right=driver.find_element_by_css_selector("a.coreSpriteRightPaginationArrow")
        right.click()
        time.sleep( random.uniform(2,7))
except:
    insta = pd.DataFrame(results)
    insta.to_csv("naver_jeju.csv",encoding='utf-8', index = False)
driver.close()